########################################
# 1. Global Setup: Define Symbols and a Potential Function
########################################


In [6]:
import random
import nltk
from nltk import CFG, Nonterminal
import symengine as se
from symengine import symbols, diff, sin, cos, exp, sympify
import sys

sys.setrecursionlimit(10000)

# Define symbols for our 1D spatial (x) and time (t) setting.
x, t = symbols('x t')

# Define a simple potential function V(x). (For example, a quadratic potential.)
def V(expr):
    return x**2



########################################
# 2. CFG for Operator Expressions
########################################

In [7]:

# This grammar produces operators by recursively combining basic functions,
# derivatives, and potential terms.
operator_grammar_str = r"""
S -> S '+' T
S -> S '-' T
S -> S '^' T
S -> T
T -> '(' S ')'
T -> T '*' F
T -> T '/' F
T -> F
F -> '-' F
F -> '(' S ')'
F -> 'sin(' S ')'
F -> 'cos(' S ')'
F -> 'grad(' S ')'
F -> 'div(' S ')'
F -> 'curl(' S ')'
F -> 'lap(' S ')'
F -> 'd/dx(' S ')'
F -> 'd/dt(' S ')'
F -> 'd2/dt2(' S ')'
F -> 'V(' S ')'
F -> 'u'
F -> 'x'
F -> 't'
F -> '0' | '1' | '2' | '3' | '4' | '5' | '6' | '7' | '8' | '9'
Nothing -> None
"""

op_cfg = CFG.fromstring(operator_grammar_str)




########################################
# 3. CFG for Manufactured Solutions
########################################


In [8]:

# This grammar generates expressions in x and t unbiasedly.
manufactured_grammar_str = r"""
S -> S '+' T
S -> S '-' T
S -> T
T -> T '*' F
T -> T '/' F
T -> F
F -> 'sin(' S ')'
F -> 'cos(' S ')'
F -> 'exp(' S ')'
F -> '(' S ')'
F -> 'x'
F -> 't'
F -> '0' | '1' | '2' | '3' | '4' | '5' | '6' | '7' | '8' | '9'
"""

manuf_cfg = CFG.fromstring(manufactured_grammar_str)



########################################
# 4. Recursive Generation with Tunable Depth
########################################

In [9]:

def generate_expression(symbol, depth, max_depth, cfg_obj):
    """
    Recursively expand the given nonterminal symbol using cfg_obj.
    If depth >= max_depth, try to choose a production whose RHS are terminals.
    """
    if not isinstance(symbol, Nonterminal):
        return symbol
    productions = cfg_obj.productions(lhs=symbol)
    if depth >= max_depth:
        terminal_prods = [p for p in productions if all(not isinstance(sym, Nonterminal) for sym in p.rhs())]
        if terminal_prods:
            production = random.choice(terminal_prods)
        else:
            production = random.choice(productions)
    else:
        production = random.choice(productions)
    return ''.join(generate_expression(sym, depth + 1, max_depth, cfg_obj) for sym in production.rhs())


def generate_operator_expression(max_depth=5):
    """Generate one operator expression string from the start symbol 'S' using op_cfg."""
    start = Nonterminal('S')
    return generate_expression(start, 0, max_depth, op_cfg)

def generate_manufactured_solution_expr(max_depth=5):
    """Generate one manufactured solution expression string from the start symbol 'S' using manuf_cfg."""
    start = Nonterminal('S')
    return generate_expression(start, 0, max_depth, manuf_cfg)



########################################
# 5. Validation of Generated Operators
########################################

In [10]:

def validate_operator_string(op_str, require_time=True, require_space=True):
    """
    Check that the operator string contains at least one time derivative (e.g. "d/dt(" or "d2/dt2(")
    and at least one spatial derivative (e.g. "lap(", "grad(", "d/dx("). Potential terms (V(...)) are optional.
    """
    time_tokens = ["d/dt(", "d2/dt2("]
    space_tokens = ["lap(", "grad(", "div(", "curl(", "d/dx("]
    has_time = any(token in op_str for token in time_tokens)
    has_space = any(token in op_str for token in space_tokens)
    if require_time and not has_time:
        return False
    if require_space and not has_space:
        return False
    return True

def generate_valid_operator(max_depth=5, attempts=100):
    """Generate a valid operator (one that meets our physical criteria) within a maximum number of attempts."""
    for _ in range(attempts):
        op = generate_operator_expression(max_depth)
        if validate_operator_string(op):
            return op
    return None


########################################
# 6. Conversion to SymEngine-Compatible Expression
########################################


In [11]:


def convert_operator_string(op_str):
    """
    Convert the operator string (with tokens like d/dt(u), lap(u), V(...)) into a string
    that SymEngine can parse.
    
    Conventions for our 1D setting:
      - d/dt(u)      -> diff(u, t)
      - d2/dt2(u)    -> diff(u, t, 2)
      - d/dx(u)      -> diff(u, x)
      - lap(u)       -> diff(u, x, 2)
      - grad(u)      -> diff(u, x)
      - div(u) and curl(u) -> diff(u, x)   (used as placeholders)
      - V(...): left as V(...)
    """
    s = op_str
    s = s.replace("d/dt(u)", "diff(u,t)")
    s = s.replace("d2/dt2(u)", "diff(u,t,2)")
    s = s.replace("d/dx(u)", "diff(u,x)")
    s = s.replace("lap(u)", "diff(u,x,2)")
    s = s.replace("grad(u)", "diff(u,x)")
    s = s.replace("div(u)", "diff(u,x)")
    s = s.replace("curl(u)", "diff(u,x)")
    s = s.replace(" ", "")
    return s


########################################
# 7. Forcing Term Computation: f = L(u)
########################################


In [12]:

def generate_forcing_term(op_str, u_str):
    """
    Convert the operator and manufactured solution strings into SymEngine expressions,
    substitute the manufactured solution into the operator, and compute f = L(u).
    """
    local_dict = {
        'x': x,
        't': t,
        'sin': sin,
        'cos': cos,
        'exp': exp,
        'diff': diff,
        # Provide the potential function V
        'V': V
    }
    # Convert manufactured solution string to a SymEngine expression.
    u_expr = sympify(u_str, locals=local_dict)
    # Convert operator string.
    op_converted = convert_operator_string(op_str)
    local_dict['u'] = u_expr  # In the operator, u will be replaced by u_expr.
    try:
        f_expr = sympify(op_converted, locals=local_dict)
        f_expr = simplify(f_expr)
    except Exception as e:
        print("Error parsing operator:", op_converted, "\nError:", e)
        f_expr = se.sympify("0")
    return f_expr, u_expr


########################################
# 8. Generate Triplets (Operator, u, f)
########################################

In [13]:


def generate_triplet(max_depth=5):
    op_str = generate_valid_operator(max_depth)
    print(op_str)
    if op_str is None:
        op_str = "lap(u)"  # Fallback.
    u_str = generate_manufactured_solution_expr(max_depth)
    f_expr, u_expr = generate_forcing_term(op_str, u_str)
    return op_str, u_str, str(f_expr)

def generate_triplets(n=10, max_depth=3):
    return [generate_triplet(max_depth) for _ in range(n)]



########################################
# 9. Main Execution
########################################


In [14]:



if __name__ == '__main__':
    num_triplets = 100
    max_depth = 2 # Adjust this to control the complexity of generated expressions.
    triplets = generate_triplets(num_triplets, max_depth)
    
    print("Generated Triplets (Operator, u, f):")
    for i, (op_str, u_str, f_str) in enumerate(triplets):
        print(f"Triplet {i+1}:")
        print("  Operator:", op_str)
        print("  u:", u_str)
        print("  f:", f_str)
        print("-" * 40)
    
    with open("operator_u_f_triplets.txt", "w") as f:
        for op_str, u_str, f_str in triplets:
            f.write("Operator: " + op_str + "\n")
            f.write("u: " + u_str + "\n")
            f.write("f: " + f_str + "\n")
            f.write("-" * 40 + "\n")


RecursionError: maximum recursion depth exceeded

In [5]:
import nltk
from nltk import CFG, Nonterminal
from nltk.parse.generate import generate

# Define a simple grammar for math expressions.
grammar = CFG.fromstring("""
S -> S '+' T
S -> S '-' T
S -> S '^' T
S -> T
T -> T '*' F
T -> T '/' F
T -> F
F -> 'u'
F -> 'x'
F -> 't'
F -> '0' | '1' | '2' | '3' | '4' | '5' | '6' | '7' | '8' | '9'
Nothing -> None
""")



In [7]:
from nltk.parse.generate import generate, demo_grammar
from nltk import CFG
# grammar = CFG.fromstring(grammar)
print(grammar)

Grammar with 21 productions (start state = S)
    S -> S '+' T
    S -> S '-' T
    S -> S '^' T
    S -> T
    T -> T '*' F
    T -> T '/' F
    T -> F
    F -> 'u'
    F -> 'x'
    F -> 't'
    F -> '0'
    F -> '1'
    F -> '2'
    F -> '3'
    F -> '4'
    F -> '5'
    F -> '6'
    F -> '7'
    F -> '8'
    F -> '9'
    Nothing -> None


In [1]:
import random
from nltk import CFG, Nonterminal

# Define the grammar in your original format.
grammar = """
S -> S '+' T
S -> S '-' T
S -> S '^' T
S -> T
T -> T '*' F
T -> T '/' F
T -> F
F -> '-' F
F -> '(' S ')'
F -> 'sin(' S ')'
F -> 'cos(' S ')'
F -> 'grad(' S ')'
F -> 'div(' S ')'
F -> 'curl(' S ')'
F -> 'lap(' S ')'
F -> 'd/dx(' S ')'
F -> 'd/dt(' S ')'
F -> 'd2/dt2(' S ')'
F -> 'V(' S ')'
F -> 'u'
F -> 'x'
F -> 't'
F -> '0' | '1' | '2' | '3' | '4' | '5' | '6' | '7' | '8' | '9'
Nothing -> None
"""

# Create the CFG object.
cfg = CFG.fromstring(grammar)

def generate_random_sentence(grammar, symbol, depth, max_depth):
    """
    Recursively expand a nonterminal symbol using the provided grammar.
    Stops expansion once depth >= max_depth by preferring productions whose RHS are all terminals.
    """
    if not isinstance(symbol, Nonterminal):
        return symbol  # Terminal reached.
    
    productions = grammar.productions(lhs=symbol)
    
    # If we've reached maximum depth, try to choose a production with only terminals.
    if depth >= max_depth:
        terminal_prods = [p for p in productions if all(not isinstance(sym, Nonterminal) for sym in p.rhs())]
        if terminal_prods:
            production = random.choice(terminal_prods)
        else:
            production = random.choice(productions)
    else:
        production = random.choice(productions)
    
    return ''.join(generate_random_sentence(grammar, sym, depth + 1, max_depth)
                   for sym in production.rhs())

def generate_operator_expression(max_depth=5):
    """Generate one operator expression string from the start symbol 'S' using a maximum recursion depth."""
    start = Nonterminal('S')
    return generate_random_sentence(cfg, start, 0, max_depth)

def count_terms(expression):
    """
    Count the number of summation and contraction operators in the expression.
    Summation operators: '+' and '-'
    Contraction (multiplicative) operators: '*' and '/'
    """
    num_sum = expression.count('+') + expression.count('-')
    num_contr = expression.count('*') + expression.count('/')
    return num_sum, num_contr

max_depth = 10  # Adjust to control complexity.
expr = generate_operator_expression(max_depth)
print("Generated Operator Expression:")
print(expr)

# num_sum, num_contr = count_terms(expr)
# print("\nCounts:")
# print("Summation terms (+ or -):", num_sum)
# print("Contraction terms (* or /):", num_contr)


Generated Operator Expression:
8*8/t/t/6*0^t/2*x*9/8/t/1/t*u/4*x*t+V(4/1*2)-d/dt(2/7/4/2*t/9-5+8-6/3*6*4)^1+V(6^0/9*5*1/5/2/0^6/5/3/9/x^3-6/6*1*t/3*5*6)*x*grad(5-5)/x+0-8


In [8]:
import random
from nltk import CFG, Nonterminal
import torch

# Define a right-recursive grammar (with epsilon productions as empty strings)
grammar = r"""
S -> T S'
S' -> '+' T S' | '-' T S' | ""
T -> F T'
T' -> '*' F T' | '/' F T' | ""
F -> '-' F
F -> '(' S ')'
F -> 'sin(' S ')'
F -> 'cos(' S ')'
F -> 'grad(' S ')'
F -> 'div(' S ')'
F -> 'curl(' S ')'
F -> 'lap(' S ')'
F -> 'd/dx(' S ')'
F -> 'd/dt(' S ')'
F -> 'd2/dt2(' S ')'
F -> 'V(' S ')'
F -> 'u'
F -> 'x'
F -> 't'
F -> DIGITS
DIGITS -> DIGIT DIGITS | DIGIT
DIGIT -> '0' | '1' | '2' | '3' | '4' | '5' | '6' | '7' | '8' | '9'
"""

# Create the CFG object
cfg = CFG.fromstring(grammar)

def generate_sentence(symbol, depth, max_depth):
    """
    Recursively expand a symbol from the CFG.
    When reaching max_depth, prefer productions that produce only terminals.
    """
    if not isinstance(symbol, Nonterminal):
        return symbol  # Terminal
    productions = cfg.productions(lhs=symbol)
    if depth >= max_depth:
        # Try to choose a production whose right-hand side are all terminals.
        terminal_prods = [p for p in productions if all(not isinstance(sym, Nonterminal) for sym in p.rhs())]
        if terminal_prods:
            production = random.choice(terminal_prods)
        else:
            production = random.choice(productions)
    else:
        production = random.choice(productions)
    return ''.join(generate_sentence(sym, depth + 1, max_depth) for sym in production.rhs())

def generate_operator_expression(max_depth=5):
    """Generate an operator expression string from the start symbol S."""
    start = Nonterminal('S')
    return generate_sentence(start, 0, max_depth)

def count_terms(expression):
    """
    Count summation operators (+ and -) and contraction operators (* and /)
    in the expression.
    """
    num_sum = expression.count('+') + expression.count('-')
    num_mult = expression.count('*') + expression.count('/')
    return num_sum, num_mult


max_depth = 5  # Tune this value to control complexity.
expr = generate_operator_expression(max_depth)
print("Generated Operator:")
print(expr)

num_sum, num_mult = count_terms(expr)
print("\nTerm Counts:")
print("Summation terms (+ or -):", num_sum)
print("Multiplication/Division terms (* or /):", num_mult)


ValueError: Unable to parse line 2: S -> T S'
Unterminated string

In [9]:
import random
from nltk import CFG, Nonterminal

# Define the grammar in a multiline string.
grammar = """
S -> S '+' T
S -> S '-' T
S -> T
T -> T '*' U
T -> T '/' U
T -> U
U -> U '^' F
U -> F
F -> 'd/dt(u)'
F -> 'd2/dt2(u)'
F -> 'd/dx(u)'
F -> 'lap(u)'
F -> 'grad(u)'
F -> 'div(u)'
F -> 'curl(u)'
F -> 'V(' S ')'
F -> 'sin(' S ')'
F -> 'cos(' S ')'
F -> 'u'
F -> 'x'
F -> 't'
F -> NUMBER
NUMBER -> DIGITS '.' DIGITS
NUMBER -> DIGITS
DIGITS -> DIGIT DIGITS | DIGIT
DIGIT -> '0' | '1' | '2' | '3' | '4' | '5' | '6' | '7' | '8' | '9'
"""

# Create the CFG object using NLTK.
cfg = CFG.fromstring(grammar)

def generate_sentence(grammar, symbol=Nonterminal('S')):
    """
    Recursively expand the given nonterminal using the productions in grammar.
    This function randomly selects a production for each nonterminal.
    """
    if not isinstance(symbol, Nonterminal):
        return symbol  # Terminal reached.
    
    productions = grammar.productions(lhs=symbol)
    production = random.choice(productions)
    return ''.join(generate_sentence(grammar, sym) for sym in production.rhs())

if __name__ == '__main__':
    # Generate a random operator expression from the start symbol 'S'.
    random_operator = generate_sentence(cfg)
    print("Generated Operator Expression:")
    print(random_operator)


RecursionError: maximum recursion depth exceeded while calling a Python object